In [ ]:
import pandas as pd
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import affine_transform

import random
import gc


In [ ]:
files = listdir("dataWeb")
data = pd.DataFrame()

Refine data

In [ ]:
for f in files:
    d = pd.read_pickle("dataWeb/"+f)
    #data = data.append(d)
    data = pd.concat([data,d], ignore_index=True)
data.columns = ["band","style","img","shape","url"]

In [ ]:
len(data)

In [ ]:
def RemoveJunk(d):
    BlackPic,WhitePic = np.zeros((200,200,3)), np.ones((200,200,3))
    d = d.drop_duplicates(subset="band") #remove duplicates
    d = d[(d["shape"]==(200, 200, 3))] #only consistent image matrix shapes (3D)
    #d = d[(d["img"]!=BlackPic)]
    return(d)

In [ ]:
data = RemoveJunk(data)

In [ ]:
len(data)

In [ ]:
plt.imshow(data["img"].loc[54], interpolation='nearest')
plt.show()

In [ ]:
data["style"].unique()

In [ ]:
StyleDictionnary ={"Doom": "Doom Metal",
                   "Drone":"Doom Metal",
                   "Sludge":"Doom Metal",
                   "Sludge Metal":"Doom Metal",
                   "Death":"Death Metal",
                   "Folk":"Folk Metal",
                   "Power":"Power Metal",
                   "Heavy":"Heavy Metal",
                   "Pagan":"Folk Metal",
                   "Viking":"Folk Metal",
                   "Neoclassical": "Symphonic",
                   "Progressive":"Progressive Metal",
                   "Deathcore":"Metalcore",
                   "Grindcore":"Metalcore",
                   "Hardcore":"Metalcore",
                   "Thrash":"Thrash Metal",
                   "Brutal Death Metal":"Death Metal",
                   "Technical Death":"Death Metal",
                   "Gothic": "Gothic Metal",
                   "Black":"Black Metal"}

CommonAdjectives = ["Atmospheric","Melodic","Blackened","Experimental"]

In [ ]:
def AggregateStyle(s, dic):
    if s in dic: 
        return(dic[s])
    else: 
        return(s)

def AggregateStyle2(s,adj):
    sl = s.split(' ',1)
    if len(sl)>1: 
        if sl[0] in adj: 
            return(sl[1])
        if sl[0] == "Progressive":        # not sure about this one (we'll see)
            return("Progressive Metal")
    else:
        return(s)
    

In [ ]:
data["style"] = data["style"].apply(AggregateStyle2, args=(CommonAdjectives,))
data["style"] = data["style"].map(lambda a: AggregateStyle(a,StyleDictionnary))

In [ ]:
len(data)

In [ ]:
data["style"].value_counts()

too mutch death, let's remove some (say 300)

In [ ]:
toRemove = data.loc[data["style"]=="Death Metal"].sample(300)

In [ ]:
data.drop(toRemove.index, axis=0,inplace=True)

In [ ]:
data["style"].value_counts()

In [ ]:
# Remove "minor" categories 
BigOnes = data["style"].value_counts()[:13]
data = data[(data["style"].isin(BigOnes.index))]
data["style"].unique()

In [ ]:
len(data)

In [ ]:
# Save in a separate pickle 
data = data.reset_index(drop=True)
#data.to_pickle("data")

## Data augmentation

In [ ]:
data = pd.read_pickle("data")
data = data.reset_index(drop=True)
data.tail(2)

In [ ]:
data["imgR"] = data["img"].map(lambda a: np.flip(a,1)) # reverse image along L (axis 1)
data["imgRed"] = data["img"].map(lambda a :np.power(a,[1.5, 1.0, 1.0])) # reduce red

In [ ]:
# https://stackoverflow.com/questions/40060786/scipy-ndimage-interpolate-affine-transform-fails

def makeReal(im): # affine transformation "flip" pics slightly
    ang1, ang2 = random.uniform(0.9,1.1),random.uniform(-0.2,0.2)

    matrix = [[ ang1, ang2,  0.0],
              [ ang2, ang1,  0],
              [   0,   0,  1]]

    return(affine_transform(im, matrix, order=1))

In [ ]:
data["imgReal"] = data["img"].map(lambda a :makeReal(a)) # 

In [ ]:
plt.imshow(data["img"].loc[50], interpolation='nearest')
plt.show()

In [ ]:
plt.imshow(data["imgR"].loc[50], interpolation='nearest')
plt.show()

In [ ]:
plt.imshow(data["imgRed"].loc[50], interpolation='nearest')
plt.show()

In [ ]:
plt.imshow(data["imgReal"].loc[50], interpolation='nearest')
plt.show()

## NN at last

In [ ]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
data = data.reindex(np.random.permutation(data.index)) # shuffle data

#X = np.concatenate([np.expand_dims(data["img"][i], axis=0) for i in  range(len(data["img"]))],axis=0) #images
#Y = pd.get_dummies(data["style"], columns=["style"]).values # labels

X1 = np.concatenate([np.expand_dims(data["img"][i], axis=0) for i in  range(len(data["img"]))],axis=0) #images
X2 = np.concatenate([np.expand_dims(data["imgR"][i], axis=0) for i in  range(len(data["imgR"]))],axis=0) #images
X3 = np.concatenate([np.expand_dims(data["imgRed"][i], axis=0) for i in  range(len(data["imgRed"]))],axis=0)#images
X4 = np.concatenate([np.expand_dims(data["imgReal"][i], axis=0) for i in  range(len(data["imgReal"]))],axis=0)
X = np.concatenate([X1,X2,X3,X4], axis =0)

dummies = pd.get_dummies(data["style"], columns=["style"])

Y = pd.get_dummies(data["style"], columns=["style"]).values # labels
Y = np.concatenate([Y,Y,Y,Y], axis =0)

X_train,X_test  = X[200:],X[:200]
Y_train,Y_test  = Y[200:],Y[:200]

print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

In [ ]:
dummies.head(5)

In [ ]:
del X1,X2,X3,X4
gc.collect()

In [ ]:
def MetalModel(input_shape):

    ### START CODE HERE ###
    X_input = Input(input_shape)
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((7, 7))(X_input)
    
    # 2D CONV
    X = Conv2D(32, (3, 3), strides = (2, 2), name = 'conv0')(X)
    #X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X= Activation('relu')(X)
    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool0')(X)
    
    # 2D CONV
    X = ZeroPadding2D((7, 7))(X)
    X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv1')(X)
    #X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X= Activation('relu')(X)
    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(13, activation='softmax', name='fc',bias_initializer='zeros')(X)
    # Create model. 
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    ### END CODE HERE ###
    return model

In [ ]:
metalModel = MetalModel((200,200,3))
metalModel.summary()

In [ ]:
metalModel.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["categorical_accuracy"])

In [ ]:
metalModel.fit(x = X_train, y = Y_train, epochs = 10,verbose=1, batch_size = 64,validation_data=(X_test, Y_test))

In [ ]:
score = metalModel.evaluate(X_test, Y_test, verbose=0)
score[1]

In [ ]:
metalModel.save("metal-model.hdf5")